In [2]:
# Import packages.
import pandas as pd
from datetime import timedelta
from sentence_transformers import SentenceTransformer
import glob

In [2]:
# Read in the data.
crypto = pd.read_pickle("../data/crypto_3_9.pkl")
tweets = pd.read_pickle("../data/tweets_3_9.pkl")

# Download the BERT model.
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# Define a function.
def get_max(min_stamp, price):
    max_stamp = min_stamp + timedelta(hours=3)
    mask = (crypto["stamp"] > min_stamp) & (crypto["stamp"] < max_stamp)
    sub_df = crypto.loc[mask]
    max_price = sub_df.price.max()

    # Create the difference.
    difference = (max_price - price) / price
    
    # Return. THIS IS IMPORTANT.
    if difference > .0025:
        return 1
    else:
        return 0

# Apply the function.
crypto["increase"] = crypto.apply(lambda x: get_max(x[0], x[2]), axis=1)

In [ ]:
# Join the data.
training_data = tweets.merge(crypto, left_on="time", right_on="stamp")
training_data = training_data.set_index("tweet_id")
training_data = training_data.loc[:, ["time", "tweet", "increase"]]
training_data["embedding"] = pd.NA

In [22]:
# BERT, in chunks.
def bert_tweets(tweet):
    """
    BERTify the Tweets.
    
    :param: tweet
    :return: encoded Tweet
    """
    
    # BERTify the chunk.
    return model.encode(str(tweet))

# Loop.
step = 3000
for min_index in range(270000, training_data.shape[0], step):
    # Create a sub DataFrame.
    max_index = min_index + step
    sub_data = training_data.iloc[min_index:max_index, :]
    sub_data["embedding"] = sub_data["tweet"].apply(bert_tweets)
    sub_data.to_pickle(f"../data/embedded/bert_{min_index}.pkl")

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

/var/folders/7l/hw5qr0qd5t5fc8_xskflkdg80000gn/T/ipykernel_7329/1206328388.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data["embedding"] = sub_data["tweet"].apply(bert_tweets)


Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

In [6]:
# Create a list of files.
read_files = list()

# Stitch the written files back together.
for file_name in glob.glob("../data/embedded/*.pkl"):
    read_files.append(pd.read_pickle(file_name))

# Concat, write the data.
stitched_data = pd.concat(read_files, axis=0)
stitched_data.to_pickle("../data/training_data.pkl")